In [2]:
''' Rewrite the basic sliders example as a standalone document with JavaScript callbacks.
Present an interactive function explorer with slider widgets.
Scrub the sliders to change the properties of the ``sin`` curve, or
type into the title text box to update the title of the plot.
Run example with python sliders-standalone-code.py
In this version, the callbacks are written as strings of JS code.
'''
import numpy as np

from bokeh.layouts import row, widgetbox
from bokeh.models import ColumnDataSource
from bokeh.models.callbacks import CustomJS
from bokeh.models.widgets import Slider, TextInput
from bokeh.plotting import figure, output_file, show

output_file("sliders-standalone.html")
# Set up data
N = 200
x = np.linspace(0, 4*np.pi, N)
y = np.sin(x)
source = ColumnDataSource(data=dict(x=x, y=y))


# Set up plot
plot = figure(plot_height=400, plot_width=400, title="my sine wave",
              tools="crosshair,pan,reset,save,wheel_zoom",
              x_range=[0, 4*np.pi], y_range=[-2.5, 2.5])

plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)


# Set up widgets
text = TextInput(title="title", value='my sine wave')
offset = Slider(title="offset", value=0.0, start=-5.0, end=5.0, step=0.1)
amplitude = Slider(title="amplitude", value=1.0, start=-5.0, end=5.0, step=0.1)
phase = Slider(title="phase", value=0.0, start=0.0, end=2*np.pi)
freq = Slider(title="frequency", value=1.0, start=0.1, end=5.1, step=0.1)


# Set up callbacks
text_callback_code ="""
    plot.title.text = text.value
"""

text.js_on_change('value', CustomJS(args=dict(text=text,plot=plot), code=text_callback_code))

sliders_callback_code="""
    const a = amplitude.value,
        b = offset.value,
        w = phase.value,
        k = freq.value,
        N = 200,
        x=source.data['x'],
        y=source.data['y']
    for(let i=0; i < N; i++) {
        x[i] = 4 * window.Math.PI * i / (N-1)
        y[i] = a * window.Math.sin(k * x[i] + w) + b
    }
    source.change.emit()
"""

slider_args = dict(source=source, amplitude=amplitude, offset=offset, phase=phase, freq=freq)

for w in [offset, amplitude, phase, freq]:
    w.js_on_change('value', CustomJS(args=slider_args,code=sliders_callback_code))


# Set up layouts and add to document
inputs = widgetbox(text, offset, amplitude, phase, freq)

show(row(inputs, plot, width=800))